In [1]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

In [2]:
import sys
sys.path.append("..")
import source.Train as train

In [3]:
x_training, y_training, encoder = train.import_data()

Data has been imported
Data has been converted from : 
 [['irrelevant']
 ['irrelevant']
 ['purchase']
 ['find-hotel']
 ['irrelevant']
 ['irrelevant']
 ['irrelevant']
 ['irrelevant']
 ['purchase']
 ['purchase']]

to one-hot encoded format : 
 [[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
example data point :  [0. 0. 0. 0. 0. 0. 0. 1.]  :  Le meilleur cabriolet hybrid moins de 5m10 minimum 400 litres de coffre ?


In [4]:
nlp = train.build_model(encoder)

find-around-me
find-flight
find-hotel
find-restaurant
find-train
irrelevant
provide-showtimes
purchase


In [5]:
train_data = train.treat_data(x_training, y_training, encoder)

In [6]:
train.train(train_data, nlp)

{'textcat': 14.678429586299899}
{'textcat': 8.63075733904947}
{'textcat': 5.351299897400361}
{'textcat': 3.422049814553295}
{'textcat': 2.810465239006148}
{'textcat': 2.017275685717112}
{'textcat': 1.539093528384534}
{'textcat': 1.434878255819806}
{'textcat': 1.0931025151019833}
{'textcat': 0.9951515330897471}
model trained and saved


Les performances de notre model sont évaluées et comparées au model exemple dans le notebook Model_testing.ipynb
Afin de l'améliorer, plusieurs technique auraient pu être utilisées :

Ici, nous utilisons un système de tokenisation, ainsi que toute une pipeline générale issue de la bibliothèque spacy. Plusieurs techniques étât de l'art, bien que plus lourdes, pourraient améliorer les performances. Il s'agit ensuite de faire un compromis entre temps/coût d'entrainement et performance attendu. Pour un bot commercial, c'est à dire un service qui n'est pas critique, il n'est probablement pas pertinent de dépenser trop d'argent pour gagner un faible pourcentage sur les métriques d'évaluation.

Le modèle peut être fortement amélioré avec des données qualitatives, et proprement prétraitées. Ici, les données sont déséquilibrées, avec des classes avec très peu de données. Pour corriger ce problème il serait intéressant de produire, ou d'acheter de nouvelles données dans les catégories les plus défavorisées, ou d'utiliser des technique d'augmentation de donnée.
D'autres mesures de prétraitement que la tokenisation existent, (vectorisation...) qui pourraient permettre si implémentées d'améliorer la performance sans augmenter drastiquement le temps de calcul.

D'un coté moins technique, l'expérience utilisateur peut bénéficier d'une amélioration continue à l'aide de métrique d'utilisation, notamment sur la satisfaction de l'utilisateur en fonction de la classe, du nombre de classes utilisées en pratique (pour savoir ou renforcer l'entrâinement spécifiquement).